---

### Imports section

In [1]:
# Let's import useful packages for this notebook

import pandas as pd
import wikipedia as wp
from geopy.geocoders import Nominatim

In [2]:
# Create an instance of Nominatim
geolocator = Nominatim(user_agent="my_app")

---

### Functions section

In [3]:
# Function to check the difference of two (2) pandas series
def is_diff(serie_1, serie_2):
    d1 = set(serie_1.unique().tolist())
    d2 = set(serie_2.unique().tolist())
    if not (d1.difference(d2)):
        print('Absolutely no difference')
    else:
        print(d1.difference(d2))

---

In [4]:
# Import and display some datasets
df_1 = pd.read_csv(r'../output/ht_airports.csv')
df_2 = pd.read_csv(r'../output/ht_dep_sup.csv')
df_3 = pd.read_csv(r'../output/ht_pop_by_dep.csv')
display(df_1.head())
display(df_2.head())
display(df_3.head())

,name,municipality,type,latitude_deg,longitude_deg,iso_country,iso_region,score
0,Toussaint Louverture International Airport,Port-au-Prince,medium_airport,18.580000,-72.292503,HT,HT-OU,975
1,Cap Haitien International Airport,Cap Haitien,medium_airport,19.733000,-72.194702,HT,HT-ND,800
2,Jacmel Airport,Jacmel,medium_airport,18.241100,-72.518501,HT,HT-SE,250
3,Port-de-Paix Airport,Port-de-Paix,medium_airport,19.933599,-72.848602,HT,HT-NO,450
4,Les Cayes Airport,Les Cayes,medium_airport,18.271099,-73.788300,HT,HT-SD,250


,Département,Chef-Lieu ou Capitale,Superficie (km²)
0,Artibonite,Les Gonaïves,4984
1,Centre,Hinche,3675
2,Grande'Anse,Jérémie,1871
3,Nippes,Miragoâne,1268
4,Nord,Cap-Haïtien,2106


,adm0code,adm0_en,adm1_en,adm1_fr,IHSI_UNFPA_2019
0,HT,Haiti,West,Ouest,5010205
1,HT,Haiti,South-East,Sud-Est,893900
2,HT,Haiti,North,Nord,1159761
3,HT,Haiti,North-East,Nord-Est,444463
4,HT,Haiti,Artibonite,Artibonite,2047152


### Displaying, renaming and merging datasets

In [5]:
df_2.columns

Index(['Département', 'Chef-Lieu ou Capitale', 'Superficie (km²)'], dtype='object')

In [6]:
df_3.columns

Index(['adm0code', 'adm0_en', 'adm1_en', 'adm1_fr', 'IHSI_UNFPA_2019'], dtype='object')

In [7]:
df_3.rename(columns={'adm1_fr':'Département'}, inplace = True)
df_3.head()

,adm0code,adm0_en,adm1_en,Département,IHSI_UNFPA_2019
0,HT,Haiti,West,Ouest,5010205
1,HT,Haiti,South-East,Sud-Est,893900
2,HT,Haiti,North,Nord,1159761
3,HT,Haiti,North-East,Nord-Est,444463
4,HT,Haiti,Artibonite,Artibonite,2047152


In [8]:
df_ht = df_2.merge(df_3[['Département','adm0code','IHSI_UNFPA_2019']], on='Département', how='right')
df_ht.rename(columns={'IHSI_UNFPA_2019':'Population'}, inplace = True)
df_ht.head()

,Département,Chef-Lieu ou Capitale,Superficie (km²),adm0code,Population
0,Artibonite,Les Gonaïves,4984,HT,2047152
1,Centre,Hinche,3675,HT,845594
2,Grande'Anse,Jérémie,1871,HT,481065
3,Nippes,Miragoâne,1268,HT,351593
4,Nord,Cap-Haïtien,2106,HT,1159761


In [9]:
df_ht.dtypes

Département              object
Chef-Lieu ou Capitale    object
Superficie (km²)          int64
adm0code                 object
Population                int64
dtype: object

### Computing a new column for density by Department

In [10]:
df_ht['Density/km²'] = (df_ht.Population / df_ht['Superficie (km²)']).apply(lambda x: round(x) + 1 )
df_ht.head()

,Département,Chef-Lieu ou Capitale,Superficie (km²),adm0code,Population,Density/km²
0,Artibonite,Les Gonaïves,4984,HT,2047152,412
1,Centre,Hinche,3675,HT,845594,231
2,Grande'Anse,Jérémie,1871,HT,481065,258
3,Nippes,Miragoâne,1268,HT,351593,278
4,Nord,Cap-Haïtien,2106,HT,1159761,552


In [11]:
df_ht.dtypes

Département              object
Chef-Lieu ou Capitale    object
Superficie (km²)          int64
adm0code                 object
Population                int64
Density/km²               int64
dtype: object

In [12]:
# Using the instance of Geopy to retrieve geographical coordinates for each Department
df_ht['address'] = df_ht['Chef-Lieu ou Capitale'] +', '+ df_ht['adm0code']
df_ht['lat'] = df_ht['address'].apply(lambda x: geolocator.geocode(x).latitude )
df_ht['lon'] = df_ht['address'].apply(lambda x: geolocator.geocode(x).longitude )
df_ht['address'] = df_ht['address'].apply(lambda x: geolocator.geocode(x).address )

df_ht.head()

,Département,Chef-Lieu ou Capitale,Superficie (km²),adm0code,Population,Density/km²,address,lat,lon
0,Artibonite,Les Gonaïves,4984,HT,2047152,412,"Les Gonaïves, Arrondissement des Gonaïves, Dép...",19.446060,-72.688434
1,Centre,Hinche,3675,HT,845594,231,"Hinche, Arrondissement de Hinche, Département ...",19.144554,-72.008753
2,Grande'Anse,Jérémie,1871,HT,481065,258,"Jérémie, Arrondissement de Jérémie, Départemen...",18.644583,-74.113918
3,Nippes,Miragoâne,1268,HT,351593,278,"Miragoâne, Arrondissement de Miragoâne, Départ...",18.444843,-73.088639
4,Nord,Cap-Haïtien,2106,HT,1159761,552,"Cap-Haïtien, Arrondissement de Cap-Haïtien, Dé...",19.759524,-72.200807


---

world = pd.read_csv(r'../input/worldcities.csv')
world.head()

dom = world[world['iso2'] == 'DO']
dom.head()

dom.shape

dom.groupby(['city', 'admin_name']).count().shape

---

In [13]:
# Use the wikipedia module to retrieve data for Dominican Republic
wp.set_lang('en')
html1 = wp.page(r"Provinces_of_the_Dominican_Republic").html()
dom1 = pd.read_html(html1, header = 0)[1]
dom1.head()

,Province,Capital,Area (km²)[5],Population[4],Density[5]
0,Azua,Azua,2531.77,298246,118
1,Baoruco,Neiba,1282.23,111269,87
2,Barahona,Barahona,1739.38,232818,134
3,Dajabón,Dajabón,1020.73,87274,86
4,Distrito Nacional,Santo Domingo,104.44,1402749,13431


display(dom.head())

In [14]:
display(dom1.head())

,Province,Capital,Area (km²)[5],Population[4],Density[5]
0,Azua,Azua,2531.77,298246,118
1,Baoruco,Neiba,1282.23,111269,87
2,Barahona,Barahona,1739.38,232818,134
3,Dajabón,Dajabón,1020.73,87274,86
4,Distrito Nacional,Santo Domingo,104.44,1402749,13431


In [15]:
dom2 = pd.read_excel(r"C:\Users\dadyf\Downloads\dom_admgz_2020.xlsx", sheet_name = None)
dom2.keys()

dict_keys(['dom_gz_adm0', 'dom_gz_adm1', 'dom_gz_adm2', 'dom_gz_adm3', 'dom_gz_adm4'])

In [16]:
display(dom2['dom_gz_adm2'].head())
print(dom2['dom_gz_adm2'].shape)

,ADM0_EN,ADM0_ES,ADM0_PCODE,ADM1_ES,ADM2_REF,ADM1_PCODE,ADM2_ES,ADM2_REF.1,ADM2_PCODE
0,Dominican Republic,República Dominicana,DO,Región Cibao Nordeste,Region Cibao Nordeste,DO01,Provincia Duarte,Provincia Duarte,DO0101
1,Dominican Republic,República Dominicana,DO,Región Cibao Nordeste,Region Cibao Nordeste,DO01,Provincia Hermanas Mirabal,Provincia Hermanas Mirabal,DO0102
2,Dominican Republic,República Dominicana,DO,Región Cibao Nordeste,Region Cibao Nordeste,DO01,Provincia María Trinidad Sánchez,Provincia Maria Trinidad Sanchez,DO0103
3,Dominican Republic,República Dominicana,DO,Región Cibao Nordeste,Region Cibao Nordeste,DO01,Provincia Samaná,Provincia Samana,DO0104
4,Dominican Republic,República Dominicana,DO,Región Cibao Noroeste,Region Cibao Noroeste,DO02,Provincia Dajabón,Provincia Dajabon,DO0201


(32, 9)


In [17]:
dom2['dom_gz_adm2']['ADM2_ES'].unique()

array(['Provincia Duarte', 'Provincia Hermanas Mirabal',
       'Provincia María Trinidad Sánchez', 'Provincia Samaná',
       'Provincia Dajabón', 'Provincia Monte Cristi',
       'Provincia Santiago Rodríguez', 'Provincia Valverde',
       'Provincia Espaillat', 'Provincia Puerto Plata',
       'Provincia Santiago', 'Provincia La Vega',
       'Provincia Monseñor Nouel', 'Provincia Sánchez Ramírez',
       'Provincia Elías Piña', 'Provincia San Juan', 'Provincia Baoruco',
       'Provincia Barahona', 'Provincia Independencia',
       'Provincia Pedernales', 'Provincia Hato Mayor',
       'Provincia Monte Plata', 'Provincia San Pedro de Macorís',
       'Distrito Nacional', 'Provincia Santo Domingo', 'Provincia Azua',
       'Provincia Peravia', 'Provincia San Cristóbal',
       'Provincia San José de Ocoa', 'Provincia El Seibo',
       'Provincia La Altagracia', 'Provincia La Romana'], dtype=object)

In [18]:
df_4 = dom2['dom_gz_adm2'][['ADM0_PCODE','ADM2_ES', 'ADM2_REF.1']]
df_4.head()

,ADM0_PCODE,ADM2_ES,ADM2_REF.1
0,DO,Provincia Duarte,Provincia Duarte
1,DO,Provincia Hermanas Mirabal,Provincia Hermanas Mirabal
2,DO,Provincia María Trinidad Sánchez,Provincia Maria Trinidad Sanchez
3,DO,Provincia Samaná,Provincia Samana
4,DO,Provincia Dajabón,Provincia Dajabon


In [19]:
df_4['ADM2_ES'] = df_4['ADM2_ES'].str.replace('Provincia ','')

C:\Users\dadyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
df_4.head()

,ADM0_PCODE,ADM2_ES,ADM2_REF.1
0,DO,Duarte,Provincia Duarte
1,DO,Hermanas Mirabal,Provincia Hermanas Mirabal
2,DO,María Trinidad Sánchez,Provincia Maria Trinidad Sanchez
3,DO,Samaná,Provincia Samana
4,DO,Dajabón,Provincia Dajabon


In [21]:
df_4['ADM2_ES']

0                     Duarte
1           Hermanas Mirabal
2     María Trinidad Sánchez
3                     Samaná
4                    Dajabón
5               Monte Cristi
6         Santiago Rodríguez
7                   Valverde
8                  Espaillat
9               Puerto Plata
10                  Santiago
11                   La Vega
12            Monseñor Nouel
13           Sánchez Ramírez
14                Elías Piña
15                  San Juan
16                   Baoruco
17                  Barahona
18             Independencia
19                Pedernales
20                Hato Mayor
21               Monte Plata
22      San Pedro de Macorís
23         Distrito Nacional
24             Santo Domingo
25                      Azua
26                   Peravia
27             San Cristóbal
28          San José de Ocoa
29                  El Seibo
30             La Altagracia
31                 La Romana
Name: ADM2_ES, dtype: object

In [22]:
df_4['address'] = df_4['ADM2_ES'] +', '+ df_4['ADM0_PCODE']

C:\Users\dadyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_4.head()

,ADM0_PCODE,ADM2_ES,ADM2_REF.1,address
0,DO,Duarte,Provincia Duarte,"Duarte, DO"
1,DO,Hermanas Mirabal,Provincia Hermanas Mirabal,"Hermanas Mirabal, DO"
2,DO,María Trinidad Sánchez,Provincia Maria Trinidad Sanchez,"María Trinidad Sánchez, DO"
3,DO,Samaná,Provincia Samana,"Samaná, DO"
4,DO,Dajabón,Provincia Dajabon,"Dajabón, DO"


In [24]:
df_4['lat'] = df_4['address'].apply(lambda x: geolocator.geocode(x).latitude )
df_4['lon'] = df_4['address'].apply(lambda x: geolocator.geocode(x).longitude )
df_4['address'] = df_4['address'].apply(lambda x: geolocator.geocode(x).address )

C:\Users\dadyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\dadyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dadyf\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [25]:
df_4.head()

,ADM0_PCODE,ADM2_ES,ADM2_REF.1,address,lat,lon
0,DO,Duarte,Provincia Duarte,"Duarte, República Dominicana",19.243456,-70.087743
1,DO,Hermanas Mirabal,Provincia Hermanas Mirabal,"Hermanas Mirabal, República Dominicana",19.408967,-70.392841
2,DO,María Trinidad Sánchez,Provincia Maria Trinidad Sanchez,"María Trinidad Sánchez, República Dominicana",19.444598,-69.984468
3,DO,Samaná,Provincia Samana,"Samaná, República Dominicana",19.264811,-69.493002
4,DO,Dajabón,Provincia Dajabon,"Dajabón, República Dominicana",19.458693,-71.602296


In [26]:
display(df_ht.head())
display(dom1.head())
display(df_4.head())

,Département,Chef-Lieu ou Capitale,Superficie (km²),adm0code,Population,Density/km²,address,lat,lon
0,Artibonite,Les Gonaïves,4984,HT,2047152,412,"Les Gonaïves, Arrondissement des Gonaïves, Dép...",19.446060,-72.688434
1,Centre,Hinche,3675,HT,845594,231,"Hinche, Arrondissement de Hinche, Département ...",19.144554,-72.008753
2,Grande'Anse,Jérémie,1871,HT,481065,258,"Jérémie, Arrondissement de Jérémie, Départemen...",18.644583,-74.113918
3,Nippes,Miragoâne,1268,HT,351593,278,"Miragoâne, Arrondissement de Miragoâne, Départ...",18.444843,-73.088639
4,Nord,Cap-Haïtien,2106,HT,1159761,552,"Cap-Haïtien, Arrondissement de Cap-Haïtien, Dé...",19.759524,-72.200807


,Province,Capital,Area (km²)[5],Population[4],Density[5]
0,Azua,Azua,2531.77,298246,118
1,Baoruco,Neiba,1282.23,111269,87
2,Barahona,Barahona,1739.38,232818,134
3,Dajabón,Dajabón,1020.73,87274,86
4,Distrito Nacional,Santo Domingo,104.44,1402749,13431


,ADM0_PCODE,ADM2_ES,ADM2_REF.1,address,lat,lon
0,DO,Duarte,Provincia Duarte,"Duarte, República Dominicana",19.243456,-70.087743
1,DO,Hermanas Mirabal,Provincia Hermanas Mirabal,"Hermanas Mirabal, República Dominicana",19.408967,-70.392841
2,DO,María Trinidad Sánchez,Provincia Maria Trinidad Sanchez,"María Trinidad Sánchez, República Dominicana",19.444598,-69.984468
3,DO,Samaná,Provincia Samana,"Samaná, República Dominicana",19.264811,-69.493002
4,DO,Dajabón,Provincia Dajabon,"Dajabón, República Dominicana",19.458693,-71.602296


In [27]:
df_4.rename(columns={'ADM0_PCODE':'adm0code', 'ADM2_ES':'Province'}, inplace = True)
df_dom = dom1.merge(df_4, on='Province', how='right')
df_dom.head()

C:\Users\dadyf\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Province,Capital,Area (km²)[5],Population[4],Density[5],adm0code,ADM2_REF.1,address,lat,lon
0,Azua,Azua,2531.77,298246,118,DO,Provincia Azua,"Azua, República Dominicana",18.576793,-70.833477
1,Baoruco,Neiba,1282.23,111269,87,DO,Provincia Baoruco,"Baoruco, República Dominicana",18.489458,-71.330847
2,Barahona,Barahona,1739.38,232818,134,DO,Provincia Barahona,"Barahona, República Dominicana",18.171481,-71.209530
3,Dajabón,Dajabón,1020.73,87274,86,DO,Provincia Dajabon,"Dajabón, República Dominicana",19.458693,-71.602296
4,Distrito Nacional,Santo Domingo,104.44,1402749,13431,DO,Distrito Nacional,"Distrito Nacional, República Dominicana",18.485258,-69.929908


In [28]:
df_dom.columns.to_list()

['Province',
 'Capital',
 'Area (km²)[5]',
 'Population[4]',
 'Density[5]',
 'adm0code',
 'ADM2_REF.1',
 'address',
 'lat',
 'lon']

In [29]:
df_dom = df_dom[['adm0code',
 'Province',
 'ADM2_REF.1',
 'Capital',
 'Area (km²)[5]',
 'Population[4]',
 'Density[5]',
 'address',
 'lat',
 'lon']]

df_dom.head()

,adm0code,Province,ADM2_REF.1,Capital,Area (km²)[5],Population[4],Density[5],address,lat,lon
0,DO,Azua,Provincia Azua,Azua,2531.77,298246,118,"Azua, República Dominicana",18.576793,-70.833477
1,DO,Baoruco,Provincia Baoruco,Neiba,1282.23,111269,87,"Baoruco, República Dominicana",18.489458,-71.330847
2,DO,Barahona,Provincia Barahona,Barahona,1739.38,232818,134,"Barahona, República Dominicana",18.171481,-71.209530
3,DO,Dajabón,Provincia Dajabon,Dajabón,1020.73,87274,86,"Dajabón, República Dominicana",19.458693,-71.602296
4,DO,Distrito Nacional,Distrito Nacional,Santo Domingo,104.44,1402749,13431,"Distrito Nacional, República Dominicana",18.485258,-69.929908


In [30]:
import requests
import json

url = "https://dr-covid-19-tracker.p.rapidapi.com/provinces"

headers = {
    'x-rapidapi-host': "dr-covid-19-tracker.p.rapidapi.com",
    'x-rapidapi-key': "ade64d90cbmshdd1da1274dfdcfdp11cb27jsn0f762048d1e8"
    }

response = requests.request("GET", url, headers=headers)

result = response.json()

In [31]:
result = response.json()
result[0]

{'country': [{'code': 'DOM',
   'name': 'Dominican Republic',
   'total_confirmed': 4680,
   'total_recovered': 363,
   'total_deaths': 226,
   'tests': 15583}]}

In [32]:
result[1].keys()

dict_keys(['provinces'])

In [33]:
result[1]['provinces'][0]

{'iso': '01',
 'name': 'Distrito Nacional',
 'code': 'DN',
 'country': 'Dominican Republic',
 'confirmed': 1198,
 'recovered': 191,
 'deaths': 30}

In [34]:
name, confirmed , recovered, deaths = [],[],[],[]

for res in result[1]['provinces']:
    name.append(res['name'])
    confirmed.append(res['confirmed'])
    recovered.append(res['recovered'])
    deaths.append(res['deaths'])
    
covid_dom = pd.DataFrame({'Province':name, 'confirmed':confirmed, 'recovered':recovered, 'deaths':deaths})
covid_dom.head()

,Province,confirmed,recovered,deaths
0,Distrito Nacional,1198,191,30
1,Azua,22,1,1
2,Bahoruco,4,1,0
3,Barahona,25,1,1
4,Dajabón,1,0,0


In [35]:
covid_dom.shape

(32, 4)

In [36]:
is_diff(covid_dom['Province'], df_dom['Province'])

{'Elias Piña', 'Bahoruco'}


In [37]:
covid_dom['Province'] = covid_dom['Province'].replace(['Bahoruco', 'Elias Piña' ], ['Baoruco', 'Elías Piña'])

In [38]:
covid_dom['Province'].unique()

array(['Distrito Nacional', 'Azua', 'Baoruco', 'Barahona', 'Dajabón',
       'Duarte', 'Elías Piña', 'El Seibo', 'Espaillat', 'Independencia',
       'La Altagracia', 'La Romana', 'La Vega', 'María Trinidad Sánchez',
       'Monte Cristi', 'Pedernales', 'Peravia', 'Puerto Plata',
       'Hermanas Mirabal', 'Samaná', 'San Cristóbal', 'San Juan',
       'San Pedro de Macorís', 'Sánchez Ramírez', 'Santiago',
       'Santiago Rodríguez', 'Valverde', 'Monseñor Nouel', 'Monte Plata',
       'Hato Mayor', 'San José de Ocoa', 'Santo Domingo'], dtype=object)

In [39]:
df_dom_final = df_dom.merge(covid_dom, on = 'Province', how = 'left')
df_dom_final.head()

,adm0code,Province,ADM2_REF.1,Capital,Area (km²)[5],Population[4],Density[5],address,lat,lon,confirmed,recovered,deaths
0,DO,Azua,Provincia Azua,Azua,2531.77,298246,118,"Azua, República Dominicana",18.576793,-70.833477,22,1,1
1,DO,Baoruco,Provincia Baoruco,Neiba,1282.23,111269,87,"Baoruco, República Dominicana",18.489458,-71.330847,4,1,0
2,DO,Barahona,Provincia Barahona,Barahona,1739.38,232818,134,"Barahona, República Dominicana",18.171481,-71.209530,25,1,1
3,DO,Dajabón,Provincia Dajabon,Dajabón,1020.73,87274,86,"Dajabón, República Dominicana",19.458693,-71.602296,1,0,0
4,DO,Distrito Nacional,Distrito Nacional,Santo Domingo,104.44,1402749,13431,"Distrito Nacional, República Dominicana",18.485258,-69.929908,1198,191,30


In [40]:
df_dom_final.shape

(32, 13)

In [43]:
df_ht.to_csv(r'../output/cleaned/df_ht.csv', index = False)
df_ht.head()

,Département,Chef-Lieu ou Capitale,Superficie (km²),adm0code,Population,Density/km²,address,lat,lon
0,Artibonite,Les Gonaïves,4984,HT,2047152,412,"Les Gonaïves, Arrondissement des Gonaïves, Dép...",19.446060,-72.688434
1,Centre,Hinche,3675,HT,845594,231,"Hinche, Arrondissement de Hinche, Département ...",19.144554,-72.008753
2,Grande'Anse,Jérémie,1871,HT,481065,258,"Jérémie, Arrondissement de Jérémie, Départemen...",18.644583,-74.113918
3,Nippes,Miragoâne,1268,HT,351593,278,"Miragoâne, Arrondissement de Miragoâne, Départ...",18.444843,-73.088639
4,Nord,Cap-Haïtien,2106,HT,1159761,552,"Cap-Haïtien, Arrondissement de Cap-Haïtien, Dé...",19.759524,-72.200807


In [44]:
df_dom_final.to_csv(r'../output/cleaned/df_dom.csv', index = False)
df_dom_final.head()

,adm0code,Province,ADM2_REF.1,Capital,Area (km²)[5],Population[4],Density[5],address,lat,lon,confirmed,recovered,deaths
0,DO,Azua,Provincia Azua,Azua,2531.77,298246,118,"Azua, República Dominicana",18.576793,-70.833477,22,1,1
1,DO,Baoruco,Provincia Baoruco,Neiba,1282.23,111269,87,"Baoruco, República Dominicana",18.489458,-71.330847,4,1,0
2,DO,Barahona,Provincia Barahona,Barahona,1739.38,232818,134,"Barahona, República Dominicana",18.171481,-71.209530,25,1,1
3,DO,Dajabón,Provincia Dajabon,Dajabón,1020.73,87274,86,"Dajabón, República Dominicana",19.458693,-71.602296,1,0,0
4,DO,Distrito Nacional,Distrito Nacional,Santo Domingo,104.44,1402749,13431,"Distrito Nacional, República Dominicana",18.485258,-69.929908,1198,191,30
